In [1]:
import tensorflow as tf
import codecs
import os
import json
import numpy as np

In [2]:
params = {
    'batch_size': 64,
    'lr' : 0.001,
    'max_sent_len': 20,
    'epochs': 500,
    'drops' : [0.1]
         }

In [3]:
saved_model_dir = '../ner_model_weight/model_conv.h5'

In [4]:
model = tf.keras.models.load_model(saved_model_dir)

In [5]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 20)]         0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 20, 32)       16000       Input[0][0]                      
__________________________________________________________________________________________________
bidirectional (Bidirectional)   (None, 20, 128)      36864       embedding[0][0]                  
__________________________________________________________________________________________________
global_average_pooling1d (Globa (None, 128)          0           bidirectional[0][0]              
_______________________________________________________________________________________

In [6]:
def extract_data(data_path):
    """
    意图识别抽取出label
    槽位识别与填充作为命名实体识别问题，对每一个字进行实体标注, ate_time', 'B-target', 'I-date_time', 'I-date_time', 'I-operation', 'I-date_time', 'I-date_time']
[ ]:
￼
​B E I O S
    """
    with codecs.open(data_path,"r",encoding="utf-8") as fp:
        data = json.load(fp)
    texts = [example['text'].replace(" ","") for example in data]
    intent_labels = [example['intent'] for example in data]
    
    slots_ners = []
    count = 0
    for example in data:
        if 'entities' in example.keys():
            text = example['text']
            ner = ['O'] * len(text)
            slots = example['entities']
            for key,val in slots.items():
                start_idx = text.find(val)
                end_idx = start_idx + len(val) -1
                if len(val) == 1:
                    ner[start_idx] = 'S-' + key
                else:
                    ner[start_idx] = 'B-' + key
                    ner[end_idx] = 'E-'+ key
                    for idx in range(start_idx+1, end_idx):
                        ner[idx] = 'I-' + key
        else:
            text = example['text']
            ner = ['O'] * len(text)
        slots_ners.append(ner)
#     print('texts len: ', len(texts))
#     print('intent_lables len: ',len(intent_labels))
#     print('slots_ners len: ', len(slots_ners))
    return texts, intent_labels, slots_ners  

In [7]:
data_path ="../dataset/data_v2.json"
max_sent_len = params["max_sent_len"]
texts, intent_labels, slots_ners = extract_data(data_path)

train_text = [d for i , d in enumerate(texts) if i % 10 != 0]
train_l = len(train_text) // params['batch_size']

train_text = train_text[:train_l*params['batch_size']]
valid_text = [d for i , d in enumerate(texts) if i % 10 == 0]
valid_l = len(valid_text) // params['batch_size']
valid_text = valid_text[:valid_l*params['batch_size']]

train_intent = [d for i , d in enumerate(intent_labels) if i % 10 != 0]
train_intent = train_intent[:train_l*params['batch_size']]
valid_intent = [d for i , d in enumerate(intent_labels) if i % 10 == 0]
valid_intent = valid_intent[:valid_l*params['batch_size']]

train_ner = [d for i , d in enumerate(slots_ners) if i % 10 != 0]
train_ner = train_ner[:train_l*params['batch_size']]
valid_ner = [d for i , d in enumerate(slots_ners) if i % 10 == 0]
valid_ner =valid_ner[:valid_l*params['batch_size']]

In [8]:
with open('../char_6.17.json', mode='r', encoding='utf-8') as f:
    dicts = json.load(f)

char2id = dicts['char2id']
id2char = dicts['id2char']
intent2id = dicts['intent2id']
id2intent = dicts['id2intent']
slot2id = dicts['slot2id']
id2slot = dicts['id2slot']

params['intent_num'] = len(intent2id)
params['slot_num'] = len(slot2id)
params['id2intent'] = id2intent
params['id2slot'] = id2slot

In [9]:
def trans2labelid(vocab, labels, max_sent_len):
    labels = [vocab[label] for label in labels]
    if len(labels) < max_sent_len:
        labels += [0] * (max_sent_len - len(labels))
    else:
        labels = labels[:max_sent_len]
    return labels

In [10]:
def read_data(txt_seqs, intent_labels, slot_ners,char2id,intent2id,slot2id,max_sent_len):
    dataset_text_labels = []
    dataset_intent_labels = []
    dataset_ner_labels = []
    
    for index in range(len(txt_seqs)):
        dataset_text_labels.append(trans2labelid(char2id,txt_seqs[index],max_sent_len))
        dataset_intent_labels.append([intent2id[intent_labels[index]]])
        dataset_ner_labels.append(trans2labelid(slot2id,slot_ners[index],max_sent_len))
    dataset_text_labels = np.array(dataset_text_labels)
    dataset_intent_labels = np.array(dataset_intent_labels)
    dataset_ner_labels = np.array(dataset_ner_labels)
    
    return dataset_text_labels, dataset_intent_labels, dataset_ner_labels 

In [11]:
valid_seq, valid_intent, valid_ner =  read_data(valid_text, valid_intent, valid_ner,char2id,intent2id,slot2id,max_sent_len) 

In [12]:
def representative_dataset():
    for data in tf.data.Dataset.from_tensor_slices(({
    "Input" : valid_seq
    },
    {
        "pre_intent":valid_intent,
        
        "pre_ner":valid_ner
    })).batch(1).take(100):
        yield [tf.dtypes.cast(data, tf.float32)]

In [16]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpph6l8_97/assets


INFO:tensorflow:Assets written to: /tmp/tmpph6l8_97/assets


ValueError: Failed to parse the model: pybind11::init(): factory function returned nullptr.

In [6]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: /tmp/tmp60c7cd8s/assets


In [7]:
save_path = '../ner_model_weight/model_conv.tflite'
open(save_path,'wb').write(tflite_quant_model)

93728